In [2]:
import tensorflow as tf
import foolbox as fb
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Visualizations will be shown in the notebook.
%matplotlib inline

# Load pickled data
import pickle
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pandas.io.parsers import read_csv


## Loading datasets
file_path = 'train.pickle'
training_file = 'train.pickle'
validation_file='valid.pickle'
testing_file = 'test.pickle'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

sign_names = read_csv("label_names.csv").values[:, 1]

#Separating features and labels
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [6]:
# Number of training examples
n_train = X_train.shape[0]
# Number of validation examples
n_validation = X_valid.shape[0]
# Number of testing examples.
n_test = X_test.shape[0]
# What's the shape of an traffic sign image?
image_shape = X_train[0].shape
# Unique classes/labels there are in the dataset.
classes, class_indices, class_counts  = np.unique(y_train, return_index=True, return_counts=True)
n_classes = len(class_counts)


print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 34799
Number of validation examples = 4410
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


In [8]:
from sklearn.utils import shuffle

## Shuffle the dataset
X_train, y_train = shuffle(X_train, y_train)

In [10]:
# Convert RGB images to grayscale for the training set
X_train_gray = np.sum(X_train/3, axis=3, keepdims=True)

# Convert RGB images to grayscale for the testing set
X_test_gray  = np.sum(X_test/3, axis=3, keepdims=True)

# Convert RGB images to grayscale for the validation set
X_validation_gray  = np.sum(X_valid/3, axis=3, keepdims=True)

In [11]:
# Normalize the training set of grayscale images
X_train_gray_norm = (X_train_gray - 32) / 32

# Normalize the test set of grayscale images
X_test_gray_norm = (X_test_gray - 32) / 32

# Normalize the validation set of grayscale images
X_validation_gray_norm = (X_validation_gray - 32) / 32

In [12]:
# Onehot Encoding the labels.
from tensorflow.keras.utils import to_categorical

# Store the original labels for future reference
y_train_unchanged = y_train
y_validation_unchanged = y_valid
y_test_unchanged = y_test

# Perform Onehot Encoding on the labels
# Convert the labels from integer representation to binary matrix representation
y_train = to_categorical(y_train)
y_validation = to_categorical(y_valid)
y_test = to_categorical(y_test)

In [13]:
model = tf.keras.models.load_model('Checkpoint_AlexNet_Epoch_50.h5')
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [21]:
loss, accuracy = model.evaluate(X_test_gray_norm, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

395/395 ━━━━━━━━━━━━━━━━━━━━ 18s 42ms/step - accuracy: 0.8762 - loss: 0.5231
Test Loss: 0.5332487225532532
Test Accuracy: 0.8771971464157104


In [18]:
import foolbox as fb
import torch
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

# Load your trained model (ensure it's a Keras model)
model = load_model('Checkpoint_AlexNet_Epoch_50.h5')
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
fmodel = fb.TensorFlowModel(model, bounds=(-0.875, 6.96875))  # Ensure bounds match normalization

# Define FGSM attack
attack = fb.attacks.FGSM()
criterion = fb.criteria.Misclassification(tf.convert_to_tensor(y_test_unchanged, dtype=tf.int32))


# Set epsilon (perturbation magnitude)
epsilon = 0.1

X_test_gray_norm_tensor = tf.convert_to_tensor(X_test_gray_norm, dtype=tf.float32)

# Generate adversarial examples
adversarial_examples, perturbed_images, success = attack(fmodel, X_test_gray_norm_tensor, criterion=criterion, epsilons=epsilon)



Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


AttributeError: `np.find_common_type` was removed in the NumPy 2.0 release. Use `numpy.promote_types` or `numpy.result_type` instead. To achieve semantics for the `scalar_types` argument, use `numpy.result_type` and pass the Python values `0`, `0.0`, or `0j`.